# Notebook for making synthetic training data from traffic sign templates
Various image processing techniques are applied to the templates to generate synthetic data. Functions are defined to apply various image processing techniques to the templates. The functions are then combined into pipelines which are applied to the templates to generate synthetic data. The synthetic data is then used to train a traffic sign classifier.

In [2]:
import os
from google.cloud import storage
from tqdm import tqdm
import os
import cv2
import numpy as np
import shutil
from multiprocessing import Pool
import cProfile
import pstats

In [3]:
train_root = 'data/train_templates'
synthetic_root = 'data/synthetic'
if not os.path.exists(synthetic_root):
    os.makedirs(synthetic_root)

In [4]:
def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # scale image so width is 1000 pixels and keep aspect ratio
    scale = 256 / image.shape[1]
    image = cv2.resize(image, (0, 0), fx=scale, fy=scale)
    return image

## Rotate and tilt images
The `rotate_image` function rotates an image by a random angle within a specified range. The `tilt_image` function tilts an image by a random angle within a specified range. The `apply_accidental_deformations` function applies accidental deformations to an image to simulate real-world scenarios where the image may be deformed due to various reasons such as dirt, scratches, or other factors.

In [ ]:
def rotate_image(image, min_angle, max_angle, std_dev=6):
    std_dev = (max_angle - min_angle) / std_dev
    angle = np.clip(np.random.normal((min_angle + max_angle) / 2, std_dev), min_angle, max_angle)
    h, w = image.shape[:2]
    channels = image.shape[2] if image.ndim == 3 else 1
    new_w = int(np.ceil(w * np.abs(np.cos(np.radians(angle))) + h * np.abs(np.sin(np.radians(angle)))))
    new_h = int(np.ceil(w * np.abs(np.sin(np.radians(angle))) + h * np.abs(np.cos(np.radians(angle)))))
    center = (w // 2, h // 2)
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    rot_mat[0, 2] += (new_w - w) / 2
    rot_mat[1, 2] += (new_h - h) / 2

    if channels != 4:
        # If no alpha channel, add one
        image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    
    result = cv2.warpAffine(image, rot_mat, (new_w, new_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
    return result

def tilt_image(image, std_dev, tilt_type='random', padding=75):
    h, w = image.shape[:2]
    channels = image.shape[2] if image.ndim == 3 else 1
    padded_h = h + 2 * padding
    padded_w = w + 2 * padding

    # Add an alpha channel if it does not exist
    if channels != 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

    # Create a new padded image with the alpha channel
    padded_image = np.zeros((padded_h, padded_w, 4), dtype=np.uint8)

    # Copy the input image into the center of the padded image
    padded_image[padding:padding + h, padding:padding + w, :] = image

    src = np.float32([[padding, padding], [padding + w, padding], [padding, padding + h], [padding + w, padding + h]])

    # Randomly determine the tilt type and calculate the angle
    # Sample from the normal distribution
    tilt_angle_side = np.abs(np.clip(np.random.normal(0, std_dev), -0.5, 0.5))  # std dev adjusted to fit range mostly within -0.5 to 0.5
    tilt_angle_front = np.clip(np.random.normal(0.5, std_dev), 0, 1)
    if tilt_type == 'random':
        tilt_type = np.random.choice(['front', 'right', 'left'])

    # Define transformations based on the tilt type
    if tilt_type == 'front':
        dst = np.float32([
            [padding + 0.3 * w * (1 - tilt_angle_front), padding],
            [padding + w - 0.3 * w * (1 - tilt_angle_front), padding],
            [padding + 0.3 * w * tilt_angle_front, padding + h],
            [padding + w - 0.3 * w * tilt_angle_front, padding + h]
        ])

    elif tilt_type == 'right':
        factor = np.sin(np.pi * tilt_angle_side) * 0.5
        dst = np.float32([
            [padding, padding - factor * h],              # Top-left stays
            [padding + w - factor * w, padding],          # Top-right moves left
            [padding, padding + h + factor * h],          # Bottom-left stays
            [padding + w - factor * w, padding + h]       # Bottom-right moves left
        ])
    elif tilt_type == 'left':
        factor = np.sin(np.pi * tilt_angle_side) * 0.5
        dst = np.float32([
            [padding + factor * w, padding],
            [padding + w, padding - factor * h],
            [padding + factor * w, padding + h],
            [padding + w, padding + h + factor * h]
        ])
    else:
        dst = src
        
    matrix = cv2.getPerspectiveTransform(src, dst)
    result = cv2.warpPerspective(padded_image, matrix, (padded_w, padded_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

    return result

## Apply accidental deformations
The `apply_accidental_deformations` function applies accidental deformations to an image to simulate real-world scenarios where the image may be deformed due to various reasons such as dirt, scratches, or other factors.

In [5]:
def apply_accidental_deformations(image, num_regions=5, size_range=(5, 20), skip_probability=0.6):
    """
    Apply accidental deformations to the image, making deformed regions transparent.

    Parameters:
        image: Input image to deform. Expected to have an alpha channel.
        num_regions: Number of deformation regions to generate.
        size_range: Tuple of (min, max) size for the deformation regions.

    Returns:
        The deformed image.
    """
    if np.random.rand() < skip_probability:
        return image
    height, width = image.shape[:2]
    channels = image.shape[2] if image.ndim == 3 else 1

    # Ensure image has an alpha channel
    if channels != 4:
        deformed_image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    else:
        deformed_image = image.copy()

    num_regions = np.random.randint(1, num_regions + 1)

    for _ in range(num_regions):
        # Generate random region
        x = np.random.randint(0, width)
        y = np.random.randint(0, height)
        size = np.random.randint(size_range[0], size_range[1])

        # Create a mask where the region will be transparent
        mask = np.zeros((height, width), dtype=np.uint8)
        cv2.circle(mask, (x, y), size, 1, thickness=-1)

        # Set alpha to 0 (fully transparent) in the deformed regions
        deformed_image[mask == 1, 3] = 0

    return deformed_image


## Add background
The `add_background` function adds a random noise background to an image. The function first crops the image to the non-transparent parts and then generates Gaussian noise for the background with random colors. The function then creates a three-channel noise background and applies the noise only where the alpha channel is 0 (transparent areas) and copies the original image data elsewhere.

In [6]:
def add_background(image):
    # Crop to the non-transparent parts of the image
    cropped_image = extract_cropped_image(image)

    # Generate Gaussian noise for the background with random colors
    h, w = cropped_image.shape[:2]
    noise_red = np.random.normal(loc=128, scale=30, size=(h, w)).astype(np.uint8)
    noise_green = np.random.normal(loc=128, scale=30, size=(h, w)).astype(np.uint8)
    noise_blue = np.random.normal(loc=128, scale=30, size=(h, w)).astype(np.uint8)

    # Create a three-channel noise background
    noise_background = np.stack((noise_blue, noise_green, noise_red), axis=-1)  # Note the order is BGR for OpenCV

    # Create an empty array for the final image with only RGB channels
    final_image = np.zeros((h, w, 3), dtype=np.uint8)

    # Apply the noise only where the alpha channel is 0 (transparent areas) and copy original image data elsewhere
    alpha_channel = cropped_image[:,:,3] / 255.0  # Normalize alpha values to range [0,1] for blending
    for i in range(3):  # Process each color channel
        final_image[:,:,i] = (alpha_channel * cropped_image[:,:,i] + (1 - alpha_channel) * noise_background[:,:,i]).astype(np.uint8)

    return final_image

def extract_cropped_image(image):
    # Extract the alpha channel
    alpha = image[:, :, 3]

    # Ensure alpha channel is in the correct format (CV_8UC1)
    alpha = alpha.astype(np.uint8)

    # Threshold the alpha channel to create a binary mask
    _, alpha_binary = cv2.threshold(alpha, 1, 255, cv2.THRESH_BINARY)

    # Find contours in the binary mask
    contours, _ = cv2.findContours(alpha_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the bounding box coordinates from the contours
        x_min, y_min, w, h = cv2.boundingRect(contours[0])
        for contour in contours:
            x, y, w_i, h_i = cv2.boundingRect(contour)
            x_max = max(x_min + w, x + w_i)
            y_max = max(y_min + h, y + h_i)
            x_min = min(x_min, x)
            y_min = min(y_min, y)
            w, h = x_max - x_min, y_max - y_min

        # Crop the image using the bounding box coordinates
        cropped_output = image[y_min:y_min+h, x_min:x_min+w, :]

        return cropped_output
    else:
        # Return an empty image with an alpha channel if no contours are found
        return np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)


In [7]:
# convert alpha to pink when no background noise is added
def convert_alpha_to_pink(image):
    """
    Convert an image with an alpha channel to a 3-channel BGR image,
    replacing transparent areas with a pink background.

    Parameters:
        image: Input image (expected to be BGRA).

    Returns:
        A BGR image where transparent areas are now pink.
    """
    if image.shape[2] == 4:  # Check if the image has an alpha channel
        image = extract_cropped_image(image)
        # Create a pink background image
        pink_background = np.ones((image.shape[0], image.shape[1], 3), dtype=np.uint8) * np.array([180, 105, 255], dtype=np.uint8)
        # Use alpha channel as a mask to combine the image with the pink background
        alpha_channel = image[:, :, 3] / 255.0
        image_rgb = image[:, :, :3]
        foreground = (image_rgb * alpha_channel[:, :, np.newaxis]).astype(np.uint8)
        background = (pink_background * (1 - alpha_channel[:, :, np.newaxis])).astype(np.uint8)
        return cv2.add(foreground, background)
    else:
        return image


## Change contrast and brightness
The `change_contrast` function changes the contrast of an image by a random factor within a specified range. The `change_brightness` function changes the brightness of an image by a random value within a specified range. The `apply_irregular_illumination` function applies irregular illumination to an image, affecting only non-transparent areas.

In [8]:
def change_contrast(image, factor_range=(0.5, 1.5)):
    factor = np.random.uniform(factor_range[0], factor_range[1])
    return cv2.convertScaleAbs(image, alpha=factor, beta=0) #beta = 0, because beta changes brightness, alpha increases/decreases contrast

def change_brightness(image, value_range=(-15, 35)):
    value = np.random.randint(value_range[0], value_range[1])
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) #convert image to the HSV (Hue, Saturation, Value) color space to manipulate value channel which represents brightness
    h, s, v = cv2.split(hsv)
    if value >= 0:
        v = cv2.add(v, value)
        v[v > 255] = 255
    else:
        v = cv2.subtract(v, abs(value))
    final_hsv = cv2.merge((h, s, v))
    image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR) #convert back to original color space
    return image

In [10]:
def apply_fast_gaussian_blur(illumination_layer, size_range):
    """ Apply Gaussian blur more efficiently using separable convolution. """
    sigma = size_range[1] / 4  # Adjust sigma to a smaller value for faster computation
    kernel_size = int(6 * sigma + 1)  # Kernel size as an odd number close to 6*sigma
    if kernel_size % 2 == 0:
        kernel_size += 1  # Ensure kernel size is odd

    # Apply separable Gaussian blur
    blur_x = cv2.GaussianBlur(illumination_layer, (kernel_size, 1), sigmaX=sigma)
    blur_final = cv2.GaussianBlur(blur_x, (1, kernel_size), sigmaX=0, sigmaY=sigma)  # Correctly specify sigmaX
    return blur_final

# Modify the main function to use the new fast Gaussian blur function
def apply_irregular_illumination(image, std_dev, intensity_range=(50, 255), size_range=(25, 100), num_spots=20):
    height, width = image.shape[:2]
    channels = image.shape[2] if image.ndim == 3 else 1

    if channels != 4:
        raise ValueError("Image must have an alpha channel (BGRA)")

    illumination_layer = np.zeros((height, width), dtype=np.uint8)
    num_spots = int(np.abs(np.clip(np.random.normal(0, std_dev), -num_spots, num_spots)))
    for _ in range(num_spots):
        x = np.random.randint(0, width)
        y = np.random.randint(0, height)
        size = np.random.randint(size_range[0], size_range[1])
        intensity = np.random.randint(intensity_range[0], intensity_range[1])
        cv2.circle(illumination_layer, (x, y), size, (intensity,), thickness=-1)

    # Use optimized Gaussian blur
    blurred_illumination = apply_fast_gaussian_blur(illumination_layer, size_range)
    blurred_illumination_bgra = cv2.cvtColor(blurred_illumination, cv2.COLOR_GRAY2BGRA)
    blurred_illumination_bgra[:, :, 3] = image[:, :, 3]

    illuminated_image = cv2.addWeighted(image, 1, blurred_illumination_bgra, 0.5, 0)
    return illuminated_image


## Apply random blur
The `apply_random_blur` function applies a random blur to an image. The function selects a random blur function from a list of blur functions and applies it to the image. The function applies a blur only 40% of the time. The different blur functions include sharpening, mean blur, median blur, Gaussian blur, and bilateral filter.

In [11]:
def sharpen_image(image, amount=9):
    # amount controls the strength of the sharpening
    kernel_sharpening = np.array([
        [0, -1, 0],
        [-1, 5, -1],
        [0, -1, 0]
    ])
    sharpened = cv2.filter2D(image, -1, kernel_sharpening)
    return sharpened

def mean_blur_image(image, kernel_range=(5,25)):
    # kernel should be odd
    kernel_size = np.random.randint(kernel_range[0], kernel_range[1])
    if kernel_size % 2 == 0:
        kernel_size += 1
    blurred = cv2.blur(image, (kernel_size, kernel_size))
    return blurred


def median_blur_image(image, kernel_range=(5,25)):
    kernel_size = np.random.randint(kernel_range[0], kernel_range[1])
    if kernel_size % 2 == 0:
        kernel_size += 1
    blurred = cv2.medianBlur(image, kernel_size)
    return blurred

def gaussian_blur_image(image, kernel_range=(5,25), sigma_range=(1, 25)):
    kernel_size = np.random.randint(kernel_range[0], kernel_range[1])
    if kernel_size % 2 == 0:
        kernel_size += 1
    sigma = np.random.uniform(sigma_range[0], sigma_range[1])
    blurred = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)
    return blurred

def bilateral_filter_image(image, diameter=15, sigmaColor=25, sigmaSpace=25):
    filtered = cv2.bilateralFilter(image, diameter, sigmaColor, sigmaSpace)
    return filtered


def insert_speckle_noise(image, variance_range=(0.02, 0.5)):
    # from normal distribution
    variance = np.random.uniform(variance_range[0], variance_range[1])
    row, col, ch = image.shape
    gauss = np.random.randn(row, col, ch) * variance
    gauss = gauss.reshape(row, col, ch)
    noisy = image + image * gauss
    return noisy

def apply_random_blur(image):
    # Define the list of blur functions
    blur_functions = [sharpen_image, mean_blur_image, median_blur_image, gaussian_blur_image, bilateral_filter_image]
    
    # Apply blur only 40% of the time
    if np.random.rand() < 0.4:  # There's a 40% chance to apply a blur
        blur_function = np.random.choice(blur_functions)
        return blur_function(image)
    else:
        return image  # Return the original image 60% of the time

## Pipeline
Setting up the pipeline to apply the image processing functions to the templates. The pipeline is then applied to the templates to generate synthetic data.

In [23]:
def apply_pipeline(image, pipeline):
    for func in pipeline:
        if func == rotate_image:
            image = func(image, -30, 30, 8)
        elif func == tilt_image:
            image = func(image, 0.1, tilt_type='random')
        elif func == apply_irregular_illumination:
            image = func(image, 4)
        elif func == change_contrast:
            image = func(image, (0.3, 1.5))
        elif func == change_contrast:
            image = func(image, (-15, 35))
        else:
            image = func(image)
    return image

### Not multi-processing, slow
Use when testing the functions or limited resources

In [24]:
def generate_synthetic_data(train_root, synthetic_root, pipelines, amount=10):
    pipeline_id = 5  # Start naming folders from 1

    for pipeline in pipelines:
        pipeline_folder = os.path.join(synthetic_root, str(pipeline_id))
        if not os.path.exists(pipeline_folder):
            os.makedirs(pipeline_folder)

        # Process each class folder
        for root, dirs, files in os.walk(train_root):
            files = [f for f in files if not f.endswith('.DS_Store')]
            if not files:
                continue

            class_name = os.path.basename(root)
            class_folder = os.path.join(pipeline_folder, class_name)
            if not os.path.exists(class_folder):
                os.makedirs(class_folder)
            
            # Calculate the number of augmentations per template to maintain balance
            num_templates = len(files)
            augmentations_per_template = max(1, amount // num_templates)

            # Generate synthetic images
            for file in files:
                template = load_image(os.path.join(root, file))
                
                # Apply each pipeline and generate synthetic data
                for i in range(augmentations_per_template):
                    synthetic = apply_pipeline(template, pipeline)
                    synthetic = convert_alpha_to_pink(synthetic)
                    synthetic_filename = f'{i}_{file}'
                    cv2.imwrite(os.path.join(class_folder, synthetic_filename), synthetic)
        print(f'Generated synthetic data for pipeline {pipeline_id}')
        pipeline_id += 1  # Increment the folder ID for the next pipeline

In [19]:


def process_file(args):
    root, file, pipeline, pipeline_folder, class_folder, augmentations_per_template = args
    template = load_image(os.path.join(root, file))
    for i in range(augmentations_per_template):
        synthetic = apply_pipeline(template, pipeline)
        synthetic = convert_alpha_to_pink(synthetic)
        synthetic_filename = f'{i}_{file}'
        cv2.imwrite(os.path.join(class_folder, synthetic_filename), synthetic)

def generate_synthetic_data(train_root, synthetic_root, pipelines, amount=50):
    pipeline_id = 1  # Start naming folders from 1

    for pipeline in tqdm(pipelines):
        pipeline_folder = os.path.join(synthetic_root, str(pipeline_id))
        if not os.path.exists(pipeline_folder):
            os.makedirs(pipeline_folder)

        args_list = []
        # Process each class folder
        for root, dirs, files in os.walk(train_root):
            files = [f for f in files if not f.endswith('.DS_Store')]
            if not files:
                continue

            class_name = os.path.basename(root)
            class_folder = os.path.join(pipeline_folder, class_name)
            if not os.path.exists(class_folder):
                os.makedirs(class_folder)

            # Calculate the number of augmentations per template to maintain balance
            num_templates = len(files)
            augmentations_per_template = max(1, amount // num_templates)

            # Collect arguments for multiprocessing
            for file in files:
                args = (root, file, pipeline, pipeline_folder, class_folder, augmentations_per_template)
                args_list.append(args)

        # Use multiprocessing to process files
        with Pool() as pool:
            pool.map(process_file, args_list)

        print(f'Generated synthetic data for pipeline {pipeline_id}')
        pipeline_id += 1  # Increment the folder ID for the next pipeline


## Pipelines with different combinations of image processing functions

In [25]:
pipelines = [
    [rotate_image, apply_clahe],
    [rotate_image, tilt_image, apply_clahe],
    [rotate_image, tilt_image, apply_clahe, add_background],
    [rotate_image, tilt_image, apply_clahe, apply_accidental_deformations, add_background],
    [rotate_image, tilt_image, apply_clahe, apply_accidental_deformations, apply_irregular_illumination, add_background],
    [rotate_image, tilt_image, apply_clahe, apply_accidental_deformations, apply_irregular_illumination, add_background, change_brightness],
    [rotate_image, tilt_image, apply_clahe, apply_accidental_deformations, apply_irregular_illumination, add_background, change_brightness, change_contrast],
    [rotate_image, tilt_image, apply_clahe, apply_accidental_deformations, apply_irregular_illumination, add_background, change_brightness, change_contrast, apply_random_blur],    
]

In [20]:
generate_synthetic_data(train_root, synthetic_root, pipelines, 1500)

  0%|          | 0/8 [02:49<?, ?it/s]


KeyboardInterrupt: 

## Multi-processing, speedy
Use when generating a large amount of synthetic data.

In [ ]:
cache = {}  # Dictionary to store final results of each pipeline for reuse

def apply_pipeline(image, pipeline):
    for func in pipeline:
        if func == rotate_image:
            image = func(image, -30, 30, 8)
        elif func == tilt_image:
            image = func(image, 0.1, tilt_type='random')
        elif func == apply_irregular_illumination:
            image = func(image, 4)
        elif func == change_contrast:
            image = func(image, (0.3, 1.5))
        else:
            image = func(image)
    return image

def process_file(args):
    root, file, pipeline, pipeline_folder, class_folder, augmentations_per_template = args
    template = load_image(os.path.join(root, file))
    
    # Check for a cached version from a previous pipeline
    cache_key = f"{root}_{file}_{len(pipeline)}"
    if cache_key in cache:
        base_image = cache[cache_key]
    else:
        base_image = template
    
    for i in range(augmentations_per_template):
        synthetic = apply_pipeline(base_image.copy(), pipeline)
        synthetic = convert_alpha_to_pink(synthetic)
        synthetic_filename = f'{i}_{file}'
        cv2.imwrite(os.path.join(class_folder, synthetic_filename), synthetic)
    
    # Cache the last modified image for this file at this pipeline stage
    cache[cache_key] = synthetic

def generate_synthetic_data(train_root, synthetic_root, pipelines, amount=50):
    pipeline_id = 1  # Start naming folders from 1

    for pipeline in tqdm(pipelines):
        pipeline_folder = os.path.join(synthetic_root, str(pipeline_id))
        if not os.path.exists(pipeline_folder):
            os.makedirs(pipeline_folder)

        args_list = []
        for root, dirs, files in os.walk(train_root):
            files = [f for f in files if not f.endswith('.DS_Store')]
            if not files:
                continue

            class_name = os.path.basename(root)
            class_folder = os.path.join(pipeline_folder, class_name)
            if not os.path.exists(class_folder):
                os.makedirs(class_folder)

            num_templates = len(files)
            augmentations_per_template = max(1, amount // num_templates)

            for file in files:
                args = (root, file, pipeline, pipeline_folder, class_folder, augmentations_per_template)
                args_list.append(args)

        with Pool() as pool:
            pool.map(process_file, args_list)

        print(f'Generated synthetic data for pipeline {pipeline_id}')
        pipeline_id += 1

# Example use
generate_synthetic_data(train_root, synthetic_root, pipelines, 1500)

 12%|█▎        | 1/8 [08:33<59:51, 513.12s/it]

Generated synthetic data for pipeline 1


 25%|██▌       | 2/8 [26:52<1:25:46, 857.71s/it]

Generated synthetic data for pipeline 2


 38%|███▊      | 3/8 [46:05<1:22:42, 992.56s/it]

...

Generated synthetic data for pipeline 5


 75%|███████▌  | 6/8 [2:12:12<53:37, 1608.53s/it]  

Generated synthetic data for pipeline 6


100%|██████████| 8/8 [3:19:27<00:00, 1495.89s/it]

Generated synthetic data for pipeline 8


## Profiling the function, for testing purposes
Some functions were very slow, so we profiled the function to identify bottlenecks and optimize the code.

In [58]:
def profile_function():
    generate_synthetic_data(train_root, synthetic_root, pipelines, 5)

profiler = cProfile.Profile()
profiler.runcall(profile_function)
stats = pstats.Stats(profiler)
stats.sort_stats('time')  
stats.print_stats()


profile_function()

100%|██████████| 1/1 [00:03<00:00,  3.34s/it]


Generated synthetic data for pipeline 1
         6302 function calls (6206 primitive calls) in 3.347 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      275    3.192    0.012    3.192    0.012 {method 'acquire' of '_thread.lock' objects}
       16    0.092    0.006    0.092    0.006 {built-in method posix.fork}
      154    0.021    0.000    0.021    0.000 {built-in method posix.waitpid}
       79    0.003    0.000    0.003    0.000 /opt/conda/lib/python3.10/site-packages/zmq/sugar/socket.py:621(send)
       16    0.003    0.000    0.098    0.006 /opt/conda/lib/python3.10/multiprocessing/popen_fork.py:62(_launch)
        7    0.003    0.000    0.003    0.000 {method 'acquire' of '_multiprocessing.SemLock' objects}
       16    0.002    0.000    0.144    0.009 /opt/conda/lib/python3.10/multiprocessing/process.py:110(start)
       16    0.002    0.000    0.003    0.000 /opt/conda/lib/python3.10/multiprocessing/process.py:80

100%|██████████| 1/1 [00:03<00:00,  3.63s/it]

Generated synthetic data for pipeline 1
